In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "report_nov6_fn_run_PF_single_theta_dual_radio_NN.csv"
data = pd.read_csv(file_path)

# Plot the heatmap
for theta_err in data["theta_err"].unique():
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
        df = data[
            (data["movement"] == movement) & (data["theta_err"] == theta_err)
        ].copy()

        # Convert 'N' to integer for sorting
        df["N"] = df["N"].astype(int)

        # Sort by 'N' as integers
        df = df.sort_values(by="N")

        # Convert 'N' back to string and set as a categorical type with ordered categories
        df["N"] = df["N"].astype(str)
        df["N"] = pd.Categorical(
            df["N"], categories=sorted(df["N"].unique(), key=int), ordered=True
        )

        df["theta_dot_err"] = df["theta_dot_err"].astype(str)

        # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
        heatmap_data = (
            df.groupby(["N", "theta_dot_err"], observed=True)
            .agg({"mse_craft_theta": "mean"})
            .reset_index()
        )

        # Pivot the data for the heatmap
        heatmap_pivot = heatmap_data.pivot(
            index="theta_dot_err", columns="N", values="mse_craft_theta"
        )

        sns.heatmap(
            heatmap_pivot,
            annot=True,
            fmt=".2f",
            cmap="YlGnBu",
            cbar_kws={"label": "Mean mse_craft_theta"},
            ax=ax,
        )
        ax.set_title(f"{movement} theta_err:{theta_err}")
        ax.set_xlabel("N")
        ax.set_ylabel("theta_dot_err")
    fig.set_tight_layout(True)

In [ ]:
file_path = "report_nov6_fn_run_EKF_single_theta_dual_radio.csv"
data = pd.read_csv(file_path)

In [ ]:
data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "report_nov6_fn_run_EKF_single_theta_dual_radio.csv"
data = pd.read_csv(file_path)

# Plot the heatmap
for p in data["p"].unique():
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
        df = data[
            (data["movement"] == movement)
            & (data["p"] == p)
            & (data["dynamic_R"] == 0.0)
        ].copy()

        # Convert 'N' to integer for sorting
        df["noise_std"] = df["noise_std"].astype(float)
        df["phi_std"] = df["phi_std"].astype(float)

        # Sort by 'N' as integers
        df = df.sort_values(by="noise_std")
        df = df.sort_values(by="phi_std")

        # Convert 'N' back to string and set as a categorical type with ordered categories
        df["noise_std"] = df["noise_std"].astype(str)
        df["noise_std"] = pd.Categorical(
            df["noise_std"],
            categories=sorted(df["noise_std"].unique(), key=float),
            ordered=True,
        )

        df["phi_std"] = pd.Categorical(
            df["phi_std"],
            categories=sorted(df["phi_std"].unique(), key=float),
            ordered=True,
        )

        # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
        heatmap_data = (
            df.groupby(["noise_std", "phi_std"], observed=True)
            .agg({"mse_craft_theta": "mean"})
            .reset_index()
        )

        # Pivot the data for the heatmap
        heatmap_pivot = heatmap_data.pivot(
            index="phi_std", columns="noise_std", values="mse_craft_theta"
        )

        sns.heatmap(
            heatmap_pivot,
            annot=True,
            fmt=".2f",
            cmap="YlGnBu",
            cbar_kws={"label": "Mean mse_craft_theta"},
            ax=ax,
        )
        ax.set_title(f"{movement} p:{p}")
        ax.set_xlabel("noise_std")
        ax.set_ylabel("phi_std")
    fig.set_tight_layout(True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "report_nov6_fn_run_PF_single_theta_single_radio_NN.csv"
data = pd.read_csv(file_path)

# Plot the heatmap
for theta_err in data["theta_err"].unique():
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
        df = data[
            (data["movement"] == movement) & (data["theta_err"] == theta_err)
        ].copy()

        # Convert 'N' to integer for sorting
        df["N"] = df["N"].astype(int)

        # Sort by 'N' as integers
        df = df.sort_values(by="N")

        # Convert 'N' back to string and set as a categorical type with ordered categories
        df["N"] = df["N"].astype(str)
        df["N"] = pd.Categorical(
            df["N"], categories=sorted(df["N"].unique(), key=int), ordered=True
        )

        df["theta_dot_err"] = df["theta_dot_err"].astype(str)

        # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
        heatmap_data = (
            df.groupby(["N", "theta_dot_err"], observed=True)
            .agg({"mse_single_radio_theta": "mean"})
            .reset_index()
        )

        # Pivot the data for the heatmap
        heatmap_pivot = heatmap_data.pivot(
            index="theta_dot_err", columns="N", values="mse_single_radio_theta"
        )

        sns.heatmap(
            heatmap_pivot,
            annot=True,
            fmt=".2f",
            cmap="YlGnBu",
            cbar_kws={"label": "Mean mse_single_radio_theta"},
            ax=ax,
        )
        ax.set_title(f"{movement} theta_err:{theta_err}")
        ax.set_xlabel("N")
        ax.set_ylabel("theta_dot_err")
    fig.set_tight_layout(True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "report_nov6_fn_run_EKF_single_theta_single_radio.csv"
data = pd.read_csv(file_path)

# Plot the heatmap
for p in data["p"].unique():
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
        df = data[
            (data["movement"] == movement)
            & (data["p"] == p)
            & (data["dynamic_R"] == 0.0)
        ].copy()

        # Convert 'N' to integer for sorting
        df["noise_std"] = df["noise_std"].astype(float)
        df["phi_std"] = df["phi_std"].astype(float)

        # Sort by 'N' as integers
        df = df.sort_values(by="noise_std")
        df = df.sort_values(by="phi_std")

        # Convert 'N' back to string and set as a categorical type with ordered categories
        df["noise_std"] = df["noise_std"].astype(str)
        df["noise_std"] = pd.Categorical(
            df["noise_std"],
            categories=sorted(df["noise_std"].unique(), key=float),
            ordered=True,
        )

        df["phi_std"] = pd.Categorical(
            df["phi_std"],
            categories=sorted(df["phi_std"].unique(), key=float),
            ordered=True,
        )

        # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
        heatmap_data = (
            df.groupby(["noise_std", "phi_std"], observed=True)
            .agg({"mse_single_radio_theta": "mean"})
            .reset_index()
        )

        # Pivot the data for the heatmap
        heatmap_pivot = heatmap_data.pivot(
            index="phi_std", columns="noise_std", values="mse_single_radio_theta"
        )

        sns.heatmap(
            heatmap_pivot,
            annot=True,
            fmt=".2f",
            cmap="YlGnBu",
            cbar_kws={"label": "Mean mse_single_radio_theta"},
            ax=ax,
        )
        ax.set_title(f"{movement} p:{p}")
        ax.set_xlabel("noise_std")
        ax.set_ylabel("phi_std")
    fig.set_tight_layout(True)